In [2]:
!pip install -r requirements.txt

INFO: pip is looking at multiple versions of together to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of together to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 245, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-p

In [1]:
!pip install langchain_community langchain-text-splitters unstructured[local-inference] pdf2image pdfminer.six langchain-together pillow_heif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.5 MB/

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.fake import FakeEmbeddings
from langchain_community.vectorstores import Vectara
from langchain_text_splitters import CharacterTextSplitter

In [6]:
from google.colab import userdata

TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')
vectara_customer_id =  userdata.get('VECTARA_CUSTOMER_ID')
vectara_corpus_id = userdata.get('VECTARA_CORPUS_ID')
vectara_api_key = userdata.get('VECTARA_API_KEY')

In [7]:
vectorstore = Vectara(
                vectara_customer_id=vectara_customer_id,
                vectara_corpus_id=vectara_corpus_id,
                vectara_api_key=vectara_api_key
            )

In [8]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [37]:
!mkdir docs
# upload sample file

In [9]:
loader = UnstructuredPDFLoader('ISB-020-U3-W-S-01-B18003-001-020.pdf', strategy='fast')
data = loader.load()

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [3]:
import json

from langchain_community.document_transformers import DoctranPropertyExtractor
from langchain_core.documents import Document

In [12]:
properties = [
    {
        "name": "document_number",
        "description": "Unique identifier for the document within its project.",
        "type": "string",
        "required": True
    },
    {
        "name": "discipline",
        "description": "The discipline associated with the document.",
        "type": "string",
        "required": True
    },
    {
        "name": "title",
        "description": "Title of the document.",
        "type": "string",
        "required": True
    },
    {
        "name": "version",
        "description": "Version number of the document.",
        "type": "integer",
        "required": True
    },
    {
        "name": "date",
        "description": "Creation date of the document.",
        "type": "string",
        "format": "date",
        "required": True
    },
    {
        "name": "author",
        "description": "Author of the document.",
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "required": True
            },
            "email": {
                "type": "string",
                "format": "email",
                "required": False
            }
        },
        "required": True
    },
    {
        "name": "related_documents",
        "description": "List of related documents.",
        "type": "array",
        "items": {
            "type": "string"
        },
        "required": False
    },
    {
        "name": "status",
        "description": "Current status of the document.",
        "type": "string",
        "enum": ["draft", "under_review", "approved", "rejected"],
        "required": True
    },
    {
        "name": "keywords",
        "description": "Keywords associated with the document.",
        "type": "array",
        "items": {
            "type": "string"
        },
        "required": False
    },
    {
        "name": "summary",
        "description": "Short summary of the document content.",
        "type": "string",
        "required": False
    }
]

In [13]:
property_extractor = DoctranPropertyExtractor(properties=properties)

ValueError: Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter.

In [4]:
from dotenv import load_dotenv

load_dotenv()

ModuleNotFoundError: No module named 'dotenv'